

```
# Copyright 2024 Google LLC

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
```



# YouTube AI Generated Live Stream: Colab

Author: Guillaume Bentaieb <br />
Instruction: this Notebook needs to be opened in Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YouTubeLabs/code-samples/blob/main/yt_ai_generated_live_stream/yt_ai_generated_live_stream.ipynb)

# Introduction

Let's have some fun with AI and YouTube !

## Objective
**The goal of this Colab** is to create a live stream fully generated via AI. For that, we will use the [YouTube Data API](https://developers.google.com/youtube/v3) to pull every X seconds the lastest chat message posted by viewers on the live stream, send that as a prompt to [Vertex AI](https://cloud.google.com/vertex-ai) to generate an image via AI, and stream that image back to YouTube.

It's probably easier to understand the concept with an example. If a user comments your live stream with: "I want to see a cute dog wearing goggles", we will get that comment and send it to Vertex AI. Vertex AI will generate an image of a cute dog wearing goggles, and we will display that image in the live stream. So basically, viewers are actually creating the content that they're watching.

Want to see that in action ? [This is an example](https://www.youtube.com/watch?v=ZaI6FmWVGWE) of a YouTube live stream generated with this very Colab.

Looks cool right? :)

## Things to know before we start
**Security:**
- <ins>Moderation</ins>. Maybe the first thing that you're thinking right now is: what if the viewers ask for something inappropriate ? The algorithm used in this Colab will use Google's [Natural Language AI API](https://cloud.google.com/natural-language) to check the text against a list of safety attributes, which includes 'harmful categories' and topics that may be considered sensitive (learn more [here](https://cloud.google.com/natural-language/docs/moderating-text)). Additionally, you can learn more about YouTube Community Guidelines [here](https://www.youtube.com/howyoutubeworks/policies/community-guidelines/) and Vertex AI's safeguards [here](https://cloud.google.com/vertex-ai/generative-ai/docs/image/responsible-ai-imagen?hl=en#safety-filters).

- <ins>YouTube Live Stream</ins>: This script will never modify the visibility of your live stream, and your livestream must be unlisted for this Colab to work. Once the live stream starts, if you want to make the stream public, you will need to manually change the visibility from the YouTube Live Control Room.

**Access:**
- Make sure you do have access to the Vertex AI API [here](https://cloud.google.com/vertex-ai/docs/generative-ai/image/generate-images)
- Make sure you have enabled live streaming on your YouTube channel. More info [here](https://support.google.com/youtube/answer/2474026)

**Difficulty:**
- This Colab requires a bit of set up as we will use many Google / YT features to make it work. Nothing to be afraid of, but please don't skip any steps and read the instruction thoroughly to make sure you can get this working !

<br/>

***

<br/>

Ready ? Let's go !!

# STEP 0: Enable GPU resources for your Colab

Before we begin: This Colab runs better with GPU resources available. For that, click on "Runtime" in Colab's menu (at the top of the page), and select "change runtime type". Keep Python as runtime type, select one of the available GPU hardware (e.g. T4 GPU), and click save. If no GPU hardware is available, then no worries, we'll run everything on CPU :)

<br/>

<sub>[Optional / PRO Tip] For even better performances, you can look into running this Colab on your local machine with CUDA compatible GPU</sub>

# STEP 1: Google Cloud Setup

To use Vertex AI, Google Natural Language AI and the YouTube Data API, we need to create a Google Cloud Project, and set it up correctly.

1. First, if it's your first time using Google Cloud, then [create a new free account](https://cloud.google.com/free) and claim your free credits, which will enable you to run this Colab free of charge for countless of hours !
2. Then, create a [Google Cloud Project](https://developers.google.com/workspace/guides/create-project).
3. When your project is created, you can then go to your [Google Cloud Project Dashboard](https://console.cloud.google.com/home), make sure the right project is selected (otherwise, use the top left dropdown menu to select your newly created project), then copy the ID of your project (Project ID), paste the value below and run the script (by clicking on the play icon that shows when you over the script):

In [ ]:
global PROJECT_ID
PROJECT_ID = ''  # @param {type:"string"}

print('Project_ID saved!')

4. If this is your first time using Google Cloud, then you don't need to activate billing as you can use your free credits. Otherwise, make sure billing is activated [here](https://console.cloud.google.com/billing) (For example, if you go live for 1h with this Colab, it'll cost you less than 5 dollars [last update: 2024-20-02, subject to variations], cf [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Google Natural Language pricing](https://cloud.google.com/natural-language/pricing), and the YouTube Data API is free of charge)

5. Enable the following APIs for your project:
- [Vertex AI API](https://console.cloud.google.com/marketplace/product/google/aiplatform.googleapis.com)
- [Google Natural Language API](https://console.cloud.google.com/marketplace/product/google/language.googleapis.com)
- [YouTube Data API](https://console.cloud.google.com/marketplace/product/google/youtube.googleapis.com)

6. Create an API Key
- [Go to the API Credentials page](https://console.cloud.google.com/apis/credentials)
- Click on "Create Credentials" at the very top of the page, and click on "API Key"
- Copy the API key once it's created, run the following script and follow the instructions (spoiler: it will ask you to paste your API key !):

In [ ]:
from getpass import getpass

global API_KEY
API_KEY = getpass('Paste your API Key here, and hit enter:')

print('API_KEY saved!')

7. Connect your Google Cloud Project to this Colab by running the 2 following scripts (please follow the given instructions for each script, and don't forget to hit the "enter" key after copy / pasting your authorization code :) ):

In [ ]:
!gcloud config set project $PROJECT_ID

In [ ]:
!gcloud auth application-default login --scopes='https://www.googleapis.com/auth/cloud-platform'

# STEP 2: Create Your YouTube Live Stream

The entire goal of this project is to live stream on YouTube, so let's create a YouTube live stream !

1. Go to [YouTube](https://www.youtube.com/) and then click the "Create" icon in the top right corner (next to your profile icon and the notification bell)
2. Select "Go Live". This will redirect you to the YouTube Live Control Room
3. In the live Control Room, click on the "Manage" tab on the left, then click on "Schedule Stream". Click on "create new" (if shown) and follow the instructions. Make sure that you set the following settings for your stream:
    - Broadcast type = Streaming Software
    - Audience = Not Made For Kids
    - Altered content = Yes (more info [here](https://support.google.com/youtube/answer/14328491))
    - Comments = on (with moderation = strict if possible)
    - Who can send messages = anyone
    - Visibility = unlisted
    - Latency = low
    - (the rest can be set as you please)
4. When you're done setting up your stream, you should end up in the Live Control Room, and the URL of the page should look like this: `https://studio.youtube.com/video/XXXXXXXXXXX/livestreaming`. `XXXXXXXXXXX` is the ID of your newly created live stream. Copy it, paste it below and run the script:

In [ ]:
global LIVE_STREAM_ID
LIVE_STREAM_ID = ''  # @param {type:"string"}

print('LIVE_STREAM_ID saved!')

5. Go back to the YouTube live Control Room (URL: `https://studio.youtube.com/video/XXXXXXXXXXX/livestreaming`, where `XXXXXXXXXXX` is the ID of your newly created live stream). In the "Stream Settings" section, you should be able to find your "Stream key" for your live stream. Copy its value, paste it below and run the script:

In [ ]:
from getpass import getpass

global LIVE_STREAM_KEY
LIVE_STREAM_KEY = getpass('Paste your Stream Key here, and hit enter:')

print('LIVE_STREAM_KEY saved!')

6. In the YouTube Live Control Room, under "Stream Settings", make also sure that:
- The parameter "Enable auto-start" is set to false
- The parameter "Enable DVR" is set to false (DVR won't work well with this stream)

# STEP 3: Python Script

Let's install the python dependencies we need, and declare the functions we will use to create Gen AI images and live stream them on YouTube.

1. Run the following script to install Python dependencies:

In [ ]:
!pip install ffmpeg-python==0.2.0

2. Run the following script to define our python functions:

In [ ]:
# IMPORT

from vertexai.preview.vision_models import Image, ImageGenerationModel
from string import ascii_letters
from PIL import Image, ImageDraw, ImageFont
from multiprocessing import Process
from googleapiclient import discovery
from google.cloud import language

import google.auth
import vertexai
import textwrap
import ffmpeg
import os
import time
import torch
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.animation import FuncAnimation
from matplotlib import colormaps

################################################################################
# Global Variables, Constants and Utils Functions
################################################################################

# VERTEX AI: location of physical computing resources
VERTEX_AI_CLOUD_LOCATION = "us-central1"

# YT DATA API: define api and scopes
YT_API_SERVICE_NAME = "youtube"
YT_API_VERSION = "v3"

# YT DATA API: global variable to store YT API service. Set later during init.
global YOUTUBE
YOUTUBE = None

# PATHs to store AI Generated Images and image animation
TEMP_IMAGE_PATH = "_temp_genimage.png"
IMAGE_PATH = "genimage.png"
TEMP_ANIMATION_PATH = "_temp_animation.mov"
ANIMATION_PATH = "animation.mov"

# Image size in pixel
IMAGE_SIZE = 1080

# Minimum time to wait between 2 image generations
MIN_IMAGE_AI_GEN_REFRESH_SECONDS = 20

# Time of animation displayed on top of GEN AI image
ANIMATION_TIME_SECONDS = 5

# Fallback message to use if no live chat message can be found
FALLBACK_LIVE_CHAT_AUTHOR = "The universe"
FALLBACK_LIVE_CHAT_TEXT = "A beautiful and empty void"
FALLBACK_LIVE_CHAT_MESSAGE = {
    "messageAuthor": FALLBACK_LIVE_CHAT_AUTHOR,
    "messageText": FALLBACK_LIVE_CHAT_TEXT,
}

# Live streams details fetched later during init
global LIVE_CHAT_ID, LIVE_CHAT_PAGE_TOKEN
LIVE_CHAT_ID = None  # YT Live Chat ID
LIVE_CHAT_PAGE_TOKEN = None  # YT Chat Pagignation Token


def truncate_string(string, width):
    """Utils function: truncate long strings"""
    if len(string) > width:
        string = string[: width - 3] + "..."
    return string


def run_in_parallel(*fns):
    """Utils function: Run multiple processes in parallel"""
    proc = []
    for fn in fns:
        p = Process(target=fn)
        p.start()
        proc.append(p)
    for p in proc:
        p.join()


def execute_youtube_api_call(api_method, **kwargs):
    """Utils function: Execute YT API endpoint and catch errors"""
    result = None
    print(
        f"Calling the YouTube API endpoint: {api_method.__name__}, " +
        "with the following parameters: ",
        kwargs)
    try:
        result = api_method(**kwargs).execute()
        print(
            f"Called YouTube API endpoint {api_method.__name__} with success !"
        )
    except Exception as e:
        print(
            "An error occured while calling the Youtube API endpoint: " +
            f"{api_method.__name__}. Error:",
            e)

    return result

################################################################################
# Functions
################################################################################


def init_youtube_api():
    """Init function: initialize YOUTUBE service"""
    print("Initializing YOUTUBE service...")

    global YOUTUBE
    YOUTUBE = discovery.build(
        YT_API_SERVICE_NAME,
        YT_API_VERSION,
        developerKey=API_KEY)


def init_vertex_ai():
    """Init function: Initialize VertexAI service"""
    print("Initializing VERTEXAI service...")

    credentials, project = google.auth.default()
    vertexai.init(
        project=project,
        location=VERTEX_AI_CLOUD_LOCATION,
        credentials=credentials)


def get_text_moderation(text):
    """Get moderation for text via Google Cloud Natural Language API"""
    print(f"Moderating text: {text} ...")

    try:
        client = language.LanguageServiceClient()
        document = language.Document(
            content=text,
            type_=language.Document.Type.PLAIN_TEXT,
        )

        print("Successfully got text moderation !")
        return client.moderate_text(document=document)

    except Exception as e:
        print(f"Could not moderate the following text: {text}.")

        return None


def get_live_chat_id():
    """Get YouTube live chat id and check live set up"""
    print("Collecting live stream information...")

    # Use YouTube API to get live stream info
    video_result = execute_youtube_api_call(
        YOUTUBE.videos().list,
        part="snippet,contentDetails,status,LiveStreamingDetails",
        id=LIVE_STREAM_ID,
    )

    if not video_result or not video_result["items"]:
        raise Exception(
            "Live stream not found. Verify that your LIVE_STREAM_ID is " +
            f"correct: {LIVE_STREAM_ID}, and its privacy is set to " +
            "'unlisted'. Stream URL: " +
            "https://studio.youtube.com/video/{LIVE_STREAM_ID}/livestreaming"
        )

    video = video_result["items"][0]

    # Verify YouTube Live Stream setup
    if video["status"]["privacyStatus"] == "public":
        raise Exception(
            "Your live stream privacy is set to public. For safety measures, " +
            "you should make sure your live stream privacy is set to " +
            "'unlisted' before you run this script here: " +
            f"https://studio.youtube.com/video/{LIVE_STREAM_ID}/livestreaming"
        )

    if video["status"]["madeForKids"]:
        raise Exception(
            "Your live stream is flagged as 'Made for kids'." +
            "Please, set your live stream as 'Not Made For kids' in the " +
            "Live Control Room: "
            f"https://studio.youtube.com/video/{LIVE_STREAM_ID}/livestreaming"
        )

    live_chat_id = video["liveStreamingDetails"]["activeLiveChatId"]

    if not live_chat_id:
        raise Exception(
            "No live chat found for your stream. Make sure live chat is " +
            "enabled for your stream here: " +
            f"https://studio.youtube.com/video/{LIVE_STREAM_ID}/livestreaming"
        )

    print("Live stream correctly set up - live chat id fetched successfully !")

    # Return live chat id
    return live_chat_id


def get_live_stream_last_chat_messages():
    """Get the last chat messages of YouTube live stream"""
    print(f"Collecting last live chat messages...")

    global LIVE_CHAT_PAGE_TOKEN

    shouldFetch = True  # Fetch until we get last chat messages
    last_chat_messages = [FALLBACK_LIVE_CHAT_MESSAGE]

    while shouldFetch:
        # Get chat messages via YouTube API
        live_chat_messages_result = execute_youtube_api_call(
            YOUTUBE.liveChatMessages().list,
            liveChatId=LIVE_CHAT_ID,
            part="snippet,authorDetails",
            maxResults=2000,
            pageToken=LIVE_CHAT_PAGE_TOKEN,
        )

        # Based on API result, continue fetching or return last chat message
        if (not live_chat_messages_result
                or not live_chat_messages_result["items"]):
            shouldFetch = False

        elif (len(live_chat_messages_result["items"]) == 2000
                and live_chat_messages_result["nextPageToken"]):
            LIVE_CHAT_PAGE_TOKEN = live_chat_messages_result["nextPageToken"]

        else:
            shouldFetch = False
            last_chat_messages = [
                {
                    "messageAuthor": m["authorDetails"]["displayName"],
                    "messageText": m["snippet"]["displayMessage"],
                }
                for m in reversed(live_chat_messages_result["items"][-15:])
            ]

    return last_chat_messages


def get_most_appropriate_chat_message(live_stream_last_chat_messages):
    """Get the most appropriate chat message from a list of chat messages"""
    print(f"Selecting most appropriate live chat message ...")

    for m in live_stream_last_chat_messages:
        moderation = get_text_moderation(
            f"{m['messageAuthor']} says: {m['messageText']}"
        )

        if moderation:
            max_moderation_confidence_score = max(
                [mc.confidence for mc in moderation.moderation_categories]
            )

            if max_moderation_confidence_score < 0.8:
                return m

    print("No appropriate message found. Using fallback chat message")

    return FALLBACK_LIVE_CHAT_MESSAGE


def generate_image_from_chat_message(chat_message):
    """Generate AI image from a chat message using VertexAI"""
    print(
        "Generating image from live chat message: " +
        f"'{chat_message['messageText']}' ..."
    )

    try:
        # Generate GEN AI image with VertexAI from chat message
        model = ImageGenerationModel.from_pretrained("imagegeneration@005")
        gen_ai_image = model.generate_images(
            prompt=chat_message["messageText"], number_of_images=1
        )[0]
        gen_ai_image.save(
            location=TEMP_IMAGE_PATH,
            include_generation_parameters=True)

        # Resize image
        temp_image = Image.open(TEMP_IMAGE_PATH)
        temp_image.thumbnail((IMAGE_SIZE, IMAGE_SIZE))
        temp_image.save(TEMP_IMAGE_PATH)

        # Insert chat author and text in generated image
        image = Image.open(TEMP_IMAGE_PATH)
        image.thumbnail((IMAGE_SIZE, IMAGE_SIZE))
        font = ImageFont.truetype(font="LiberationMono-Regular.ttf", size=50)
        avg_char_width = sum(
            font.getbbox(char)[2] for char in ascii_letters
        ) / len(ascii_letters)
        max_char_count = int(image.size[0] * 0.618 / avg_char_width)
        text = textwrap.fill(
            text=truncate_string(
                f"From: {chat_message['messageAuthor']} - " +
                f"{chat_message['messageText']}",
                250,
            ),
            width=max_char_count,
        )

        draw = ImageDraw.Draw(im=image)
        draw.text(
            xy=(image.size[0] / 2, image.size[1] / 2),
            text=text,
            font=font,
            fill="#000000",
            anchor="mm",
        )

        # save as temp image and then rename image to IMAGE_PATH
        # (directly saving as IMAGE_PATH does not work well with ffmpeg stream)
        image.save(TEMP_IMAGE_PATH, optimize=True, quality=90)
        os.rename(TEMP_IMAGE_PATH, IMAGE_PATH)

        print("Image successfully generated !")
    except Exception as e:
        print(
            "failed to generate an AI image - reusing old image instead. " +
            "Error: ",
            e)

def create_video_animation():
    """Create an animation to display on top of Gen AI Image"""
    print("Creating video animation...")

    # Init Matplotlib plot
    px = 1/plt.rcParams['figure.dpi']
    fig, ax = plt.subplots(figsize=(IMAGE_SIZE*px, IMAGE_SIZE*px))
    fig.subplots_adjust(
        left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)
    fig.patch.set_alpha(0.0)
    ax.set_xlim([0, 1080*px])
    ax.set_ylim([-180*px, 900*px])
    plt.axis('off')

    # Create animation
    Nx = 50
    Nt = 200
    x = np.linspace(0, 1080*px, Nx)
    t = np.linspace(1, 25, Nt)
    X, T = np.meshgrid(x, t)

    S = -np.abs(np.linspace(-1, 1, Nt)) + 1
    F1 = 0.2*np.multiply(np.cos(T - X), S[:, None])
    F2 = 0.1*np.multiply(np.sin(2*T)*np.cos(X), S[:, None])

    frame_number = len(t)-1
    C = colormaps['coolwarm'](
        np.r_[:frame_number:2, frame_number:0:-2]/frame_number)

    fill = ax.fill_between(x, F1[0, :], F2[0, :])

    def animate(i):
        path = fill.get_paths()[0]
        verts = path.vertices
        verts[1:Nx+1, 1] = F1[i, :]
        verts[Nx+2:-1, 1] = F2[i, :][::-1]
        fill.set_color(C[i, :])

    anim = FuncAnimation(
        fig,
        animate,
        interval=ANIMATION_TIME_SECONDS * 1000 / frame_number,
        frames=frame_number)

    # Save animation as temporary and clear figure and plot
    anim.save(
        TEMP_ANIMATION_PATH,
        codec="png",
        savefig_kwargs={"transparent": True, "facecolor": "none"},
    )
    fig.clear()
    plt.close()

    # Add silent audio to animation video
    video_input = ffmpeg.input(TEMP_ANIMATION_PATH)
    audio_input = ffmpeg.input("anullsrc", format="lavfi")

    ffmpeg.output(
        video_input,
        audio_input,
        ANIMATION_PATH,
        vcodec="copy",
        acodec="aac",
        s=f"{IMAGE_SIZE}x{IMAGE_SIZE}",
        shortest=None
    ).global_args("-y").run()

    print("Video animation successfully created !")

def send_rtmp_image_stream_to_youtube():
    """Create and sends an RTMP stream to YouTube from AI Generated image"""
    print(f"Sending live stream to YouTube via RTMP...")

    image_input = None
    video_input = None
    vcodec = None
    preset = None

    if (torch.cuda.is_available()):
        image_input = ffmpeg.input(
            IMAGE_PATH, loop=1, framerate=0.25, f="image2",
            hwaccel="cuda", hwaccel_output_format="cuda")
        video_input = ffmpeg.input(
            ANIMATION_PATH, stream_loop=-1,
            hwaccel="cuda", hwaccel_output_format="cuda")
        vcodec = "h264_nvenc"
        preset = "fast"
    else:
        image_input = ffmpeg.input(
            IMAGE_PATH, loop=1, framerate=0.25, f="image2")
        video_input = ffmpeg.input(
            ANIMATION_PATH, stream_loop=-1)
        vcodec = "libx264"
        preset = "ultrafast"

    (
        ffmpeg
        .filter(image_input, 'fps', fps=25, round='up')
        .overlay(video_input)
        .output(
            f"rtmp://a.rtmp.youtube.com/live2/{LIVE_STREAM_KEY}",
            format="fifo",
            fifo_format="flv",
            map="1:a",
            drop_pkts_on_overflow=1,
            attempt_recovery=1,
            recovery_wait_time=1,
            acodec="aac",
            vcodec=vcodec,
            preset=preset,
            r=25,
            g=50,
            video_bitrate="6M",
            minrate="5M",
            maxrate="7M",
            bufsize="12M",
            timeshift=3,
            queue_size=6000
        ).run()
    )


################################################################################
# Main Processes (will be run in parallel)
################################################################################


def generate_image_process():
    """Process that continously creates new images based on new chat messages"""
    while True:
        start = time.time()
        live_stream_last_chat_messages = get_live_stream_last_chat_messages()
        chat_message = get_most_appropriate_chat_message(
            live_stream_last_chat_messages)
        generate_image_from_chat_message(chat_message)
        end = time.time()

        generate_new_image_sec = end - start
        print(f"Generated new image in: {generate_new_image_sec}s")

        if(generate_new_image_sec < MIN_IMAGE_AI_GEN_REFRESH_SECONDS):
            wait_sec = MIN_IMAGE_AI_GEN_REFRESH_SECONDS - generate_new_image_sec
            print(f"Waiting {wait_sec} seconds before generating new image")
            time.sleep(wait_sec)


def stream_image_to_youtube_process():
    """Process that continuously sends an RTMP stream to YT from GEN AI image"""
    send_rtmp_image_stream_to_youtube()

3. Execute the final script that will generate images from the live chat messages and send them continuously to YouTube:

In [ ]:
# MAIN FUNCTION

def main():
    global LIVE_CHAT_ID

    # Init YoutTube and VertexAI service
    init_youtube_api()
    init_vertex_ai()

    # Get Live Stream details
    LIVE_CHAT_ID = get_live_chat_id()

    # Create animation and 1st AI image from fallback message
    create_video_animation()
    generate_image_from_chat_message(FALLBACK_LIVE_CHAT_MESSAGE)

    # Run processes to continuously generate images and send them to YouTube
    run_in_parallel(stream_image_to_youtube_process, generate_image_process)

if __name__ == "__main__":
    main()

### And that's it !

If you go back to YouTube Studio (`https://studio.youtube.com/video/XXXXXXXXXXX/livestreaming` where XXXXXXXXXXX is your live stream ID), you should see images being sent to your live stream. You can stop sending images whenever you please by stopping the script above (click on the "stop" icon on the left of the script). If you want, you can decide to go live by clicking on the "Go Live" button in the live control room (but please, remember that this script is experimental). Please also monitor the Colab as it may [stop due to inactivity](https://research.google.com/colaboratory/faq.html#idle-timeouts)